In [14]:
from pybit.unified_trading import HTTP
import json
import time
import numpy as np
import pandas as pd
from datetime import datetime
import sqlite3
import requests
import os
from pycoingecko import CoinGeckoAPI
from tqdm import tqdm

# Bybit 資料下載

## 期現貨歷史資料下載

In [9]:
def get_all_bybit_usdt_spot_symbols():
    url = "https://api.bybit.com/v5/market/instruments-info?category=spot"
    resp = requests.get(url).json()
    symbols = [item["symbol"] for item in resp["result"]["list"]]
    return [s for s in symbols if s.endswith("USDT")]

def get_price(symbols, categories, start_date, end_date, database, interval = 1):
    
    if not symbols:
        symbols = get_all_bybit_usdt_spot_symbols()

    if not categories:
        categories = ['spot', 'linear']

    for symbol in symbols:
        for category in categories:
            try:
                start_date_ts = int(start_date.timestamp() * 1000)
                end_date_ts = int(end_date.timestamp() * 1000)
                
                klines_data = []
                first_kline_timestamp = None

                while True:
                    klines = session.get_kline(
                        category = category,
                        symbol = symbol,
                        interval = f"{interval}",
                        start = start_date_ts,
                        end = end_date_ts,
                        limit = 1000
                    )

                    klines = klines['result']['list']
                    
                    end_timestamp = int(klines[-1][0])

                    klines_data.extend(klines)

                    first_kline_timestamp = end_timestamp

                    end_date_ts = first_kline_timestamp - 1
                    
                    if len(klines) < 1000:
                        break
                    
                    time.sleep(0.5)

                header = ['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Turnover']

                df = pd.DataFrame(reversed(klines_data), columns = header)

                df['Timestamp'] = pd.to_numeric(df['Timestamp'], errors = 'coerce')
                df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit = 'ms', utc = True)
                df['Timestamp'] = df['Timestamp'].dt.tz_convert('Asia/Taipei')
                df.set_index('Timestamp', inplace = True)

                save_dir = os.path.join(database, category)
                os.makedirs(save_dir, exist_ok = True)
                df.to_csv(os.path.join(save_dir, f"{symbol}.csv"))
                
            except Exception as e:
                print(symbol, category, e)

## 資金費率下載

In [10]:
def get_all_bybit_perp_symbols():
    url = "https://api.bybit.com/v5/market/instruments-info?category=linear"
    resp = requests.get(url).json()
    symbols = [item["symbol"] for item in resp["result"]["list"]]
    return [s for s in symbols if s.endswith("USDT")]

def get_funding_rate(symbols, start_date, end_date, database):
    
    if not symbols:
        symbols = get_all_bybit_perp_symbols()
        
    for symbol in symbols:
        try:
            start_date_ts = int(start_date.timestamp() * 1000)
            end_date_ts = int(end_date.timestamp() * 1000)

            funding_rate_data = []

            while True:
                funding_rate = session.get_funding_rate_history(
                    category = "linear",
                    symbol = symbol,
                    startTime = start_date_ts,
                    endTime = end_date_ts,
                    limit = 200
                )

                funding_rate = funding_rate["result"]["list"]
                funding_rate_data.extend(funding_rate)

                end_date_ts = int(funding_rate[-1]["fundingRateTimestamp"]) - 1

                if len(funding_rate) < 200:
                    break
                    
            df = pd.DataFrame(reversed(funding_rate_data))
            df["fundingRateTimestamp"] = pd.to_numeric(df["fundingRateTimestamp"], errors = "coerce")
            df["fundingRateTimestamp"] = pd.to_datetime(df["fundingRateTimestamp"], unit = "ms", utc = True)
            df["fundingRateTimestamp"] = df["fundingRateTimestamp"].dt.tz_convert("Asia/Taipei")
            df.set_index("fundingRateTimestamp", inplace = True)

            save_dir = os.path.join(database, "funding_rate")
            os.makedirs(save_dir, exist_ok = True)
            df.to_csv(os.path.join(save_dir, f"{symbol}.csv"))
            
        except Exception as e:
            print(e)

## 板塊分類

In [16]:
def fetch_sector_map(base_path):
    """
    建立 sector_map.json，分類 spot、linear 和 funding_rate 資料夾中已有的幣種。
    
    只使用 CoinGecko 的分類 API，將未分類的標的歸入 "Others" 類別。
    
    Parameters:
    - base_path (str): 基礎路徑，應包含 spot/、linear/ 和 funding_rate/ 資料夾。
    """

    categories_to_process = ["spot", "linear", "funding_rate"]
    meta_dir = os.path.join(base_path, "metadata")
    os.makedirs(meta_dir, exist_ok=True)

    cg = CoinGeckoAPI()

    for category_type in categories_to_process:
        category_dir = os.path.join(base_path, category_type)
        if not os.path.exists(category_dir):
            print(f"⚠ {category_type} 資料夾不存在：{category_dir}")
            continue

        existing_symbols = set(f.replace(".csv", "") for f in os.listdir(category_dir) if f.endswith("USDT.csv"))
        if not existing_symbols:
            print(f"⚠ {category_type} 資料夾中沒有任何 USDT 幣種資料")
            continue

        try:
            categories = cg.get_coins_categories()
        except Exception as e:
            print(f"⚠ 無法取得 CoinGecko 分類資料：{e}")
            return

        sector_map = {}

        for category in tqdm(categories, desc = f"{category_type} 分類處理中"):
            cat_id = category['id']
            cat_name = category['name']
            try:
                coins = cg.get_coins_markets(vs_currency = 'usd', category = cat_id)
                symbols = [coin['symbol'].upper() + "USDT" for coin in coins]
                matched = list(set(symbols) & existing_symbols)
                if matched:
                    sector_map[cat_name] = {
                        "id": cat_id,
                        "symbols": sorted(matched)
                    }
                time.sleep(1.2)
            except Exception as e:
                print(f"無法抓取分類 {cat_name}: {e}")

        categorized = set(sym for v in sector_map.values() for sym in v["symbols"])
        uncategorized = sorted(list(existing_symbols - categorized))

        if uncategorized:
            sector_map["Others"] = {
                "id": "uncategorized",
                "symbols": uncategorized
            }

        sector_file = os.path.join(meta_dir, f"{category_type}_sector_map.json")
        with open(sector_file, "w") as f:
            json.dump(sector_map, f, indent = 2)

In [ ]:
with open('/Users/xinc./Documents/GitHub/Quant/data/api_key.json', 'r') as file:
    api_config = json.load(file)

api_key = api_config['api_key']
api_secret = api_config['api_secret']

session = HTTP(
    testnet = False,
    api_key = api_key,
    api_secret = api_secret
)

start_date = datetime(2022, 1, 1)
end_date = datetime.now()

symbols = [] # 空的為全部
categories = [] # 空的為現貨+永續
interval = 60 # 分鐘
base = "/Users/xinc./Documents/GitHub/Quant/data/crypto_database"

get_price(symbols, categories, start_date, end_date, base, interval)
get_funding_rate(symbols, start_date, end_date, base)
fetch_sector_map(base)

KeyboardInterrupt: 

# 學長code

#### Binance期貨資料下載

In [ ]:
all_symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

for ff in all_symbols:
    # config_logging(logging, logging.DEBUG)
    Futures_client = API()

    start_time_date = datetime(2025, 1, 1)
    end_time_date = datetime().now()
    klines_raw = []

    while start_time_date < end_time_date:
        start_time_ts = int(start_time_date.timestamp()) * 1000
        klines = Futures_client.futures_klines(symbol = ff, interval = "4h", startTime = start_time_ts, limit = 1000)
        klines_raw.extend(klines)
        last_klines = klines[-1][0]
        start_time_date = last_klines + 1
        time.sleep(15)

    # preprocessing 1
    headers = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', '#trade', 'taker_buy_vol', 'taker_buy_quote_vol', 'ignore']
    klines_df = pd.DataFrame(klines_raw, columns = headers, dtype = float)
    klines_df.drop(columns = ['ignore', 'quote_vol', 'taker_buy_quote_vol'], inplace = True)

    # Convert timestamps to datetime objects
    klines_df['open_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['open_time']))
    klines_df['close_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['close_time']))
    # preprocessing 4
    first_day_of_week = 3 # the day of the week of the very first day of the interval
    week_list = list((klines_df.index + first_day_of_week * 96) // 96 % 7)
    klines_df['week'] = week_list

    # Save dataframe to CSV file
    klines_df.to_csv(ff + '_期貨.csv', index=False)


#### Binance現貨資料下載

In [ ]:
all_symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

for ss in all_symbols:
    # config_logging(logging, logging.DEBUG)
    Spot_client = Spot(timeout = 5)

    start_time_date = datetime(2022,1,1)
    end_time_date = datetime(2024,5,10)
    klines_raw = []

    while start_time_date < end_time_date:
        start_time_ts = datetime.timestamp(start_time_date)
        klines_raw = klines_raw + Spot_client.klines(symbol = ss, interval = "4h", startTime = int(start_time_ts)*1000, limit = 500)
        start_time_date = start_time_date + timedelta(hours=4*500)
        time.sleep(30)

    # preprocessing 1
    headers = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', '#trade', 'taker_buy_vol', 'taker_buy_quote_vol', 'ignore']
    klines_df = pd.DataFrame(klines_raw, columns = headers, dtype=float)
    klines_df.drop(columns=['ignore', 'quote_vol', 'taker_buy_quote_vol'], inplace=True)

    # Convert timestamps to datetime objects
    klines_df['open_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['open_time']))
    klines_df['close_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['close_time']))
    # preprocessing 4
    first_day_of_week = 3 # the day of the week of the very first day of the interval
    week_list = list((klines_df.index + first_day_of_week * 96) // 96 % 7)
    klines_df['week'] = week_list

    # Save dataframe to CSV file
    klines_df.to_csv(ss + '_現貨.csv', index=False)

#### Binance 資金費率下載

In [ ]:
# 填入你的 Binance API key 和 secret
api_key = 'SOvKFyvNT6rC6OZvJ569D8LxUCWW4PVQOliJKfZ0mfRC0R85W6KZVQnAEgzNWL9e'
api_secret = 'OvtqqZCslAbfQvmt8SjrgASaJawebaNYa7sIgPBkFiBNCaiTS5RdAzXW7jR9OMnL'

client = Client(api_key, api_secret)

# 定義時間區段
start_str = '2022-01-01'
end_str = '2024-05-17'
start_date = datetime.strptime(start_str, '%Y-%m-%d')
end_date = datetime.strptime(end_str, '%Y-%m-%d')

# 定義所需要的幣種
symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

# 定義每次請求所需要的時間段(避免一次要求太多)
time_interval = timedelta(days=15)

# 獲取數據並存到csv檔案裡面
for symbol in symbols:
    funding_rates = []
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + time_interval, end_date)
        funding_rate_data = client.futures_funding_rate(
            symbol=symbol,
            startTime=int(current_start.timestamp() * 1000),
            endTime=int(current_end.timestamp() * 1000)
        )
        funding_rates.extend(funding_rate_data)
        current_start = current_end
    
    # 保存數據到各自的csv檔案
    csv_file = f'{symbol}_funding_rates.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['open_time', 'fundingRate'])
        for rate in funding_rates:
            # 轉換時間為UTC+0
            funding_time_utc = datetime.utcfromtimestamp(rate['fundingTime'] / 1000).strftime('%Y/%m/%d %H:%M')
            writer.writerow([funding_time_utc, rate['fundingRate']])
    
    print(f"{symbol} 資金費率數據已保存到 {csv_file}")